<a href="https://colab.research.google.com/github/sophiagemanuel/Data-Analysis-Student-Success/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dependencies
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 1s (177 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to prov

In [ ]:
# Loading the Dataset
# Reading in our CSV file
df = pd.read_csv("StudentsPerformance.csv")
# Printing out the first 10 rows of our data
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [ ]:
df.columns

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'math score', 'reading score',
       'writing score'],
      dtype='object')

In [ ]:
# Check for missing values
df.isnull().sum()

,0
gender,0
race/ethnicity,0
parental level of education,0
lunch,0
test preparation course,0
math score,0
reading score,0
writing score,0


In [ ]:
# Encode categorical variables
categorical_columns = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']
df = pd.get_dummies(df, columns=categorical_columns)

In [ ]:
# Normalize numerical features
# A American Indian/Alaska Native, B Asian, C Black or African American, D Native Hawaiian or other Pacific Islander, E White
from sklearn.preprocessing import StandardScaler
numerical_columns = ['math score', 'reading score', 'writing score']
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

df.head()

,math score,reading score,writing score,gender_female,gender_male,race/ethnicity_group A,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E,parental level of education_associate's degree,parental level of education_bachelor's degree,parental level of education_high school,parental level of education_master's degree,parental level of education_some college,parental level of education_some high school,lunch_free/reduced,lunch_standard,test preparation course_completed,test preparation course_none
0,0.390024,0.193999,0.391492,True,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,True
1,0.192076,1.427476,1.313269,True,False,False,False,True,False,False,False,False,False,False,True,False,False,True,True,False
2,1.577711,1.770109,1.642475,True,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,True
3,-1.259543,-0.833899,-1.583744,False,True,True,False,False,False,False,True,False,False,False,False,False,True,False,False,True
4,0.653954,0.605158,0.457333,False,True,False,False,True,False,False,False,False,False,False,True,False,False,True,False,True


In [ ]:
# Create an average score feature
df['average score'] = df[['math score', 'reading score', 'writing score']].mean(axis=1)

# Define the target variable (e.g., students with an average score above 70 are considered successful)
df['success'] = df['average score'] > 70

# Drop the original score columns if necessary
df = df.drop(columns=numerical_columns)

# Display the modified dataframe
df.head()

,gender_female,gender_male,race/ethnicity_group A,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E,parental level of education_associate's degree,parental level of education_bachelor's degree,parental level of education_high school,parental level of education_master's degree,parental level of education_some college,parental level of education_some high school,lunch_free/reduced,lunch_standard,test preparation course_completed,test preparation course_none,average score,success
0,True,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,True,0.325171,False
1,True,False,False,False,True,False,False,False,False,False,False,True,False,False,True,True,False,0.977607,False
2,True,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,True,1.663432,False
3,False,True,True,False,False,False,False,True,False,False,False,False,False,True,False,False,True,-1.225729,False
4,False,True,False,False,True,False,False,False,False,False,False,True,False,False,True,False,True,0.572148,False


In [ ]:

# Save the DataFrame to a new CSV file
df.to_csv('processed_student_performance.csv', index=False)

# Check if the file was saved successfully
print("File saved:", os.path.exists('processed_student_performance.csv'))

File saved: True


In [ ]:
# With Google Colab, download the file
from google.colab import files
files.download('processed_student_performance.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>